# This is The Title of Notebook
### Purpose
In this notebook I will create an example header and show how to structure the notebooks for easy readaility and documentation. 

In short, we are trying to follow the conventions of Cookie Cutter Data Science: https://drivendata.github.io/cookiecutter-data-science/ 


### Author: 
Jacob Ellena
### Date: 
2020-07-30
### Update Date: 
2020-07-30

### Inputs 
1.3-rec-connecting-fips-ecosystem-data.txt -  
Example
cbc_effort_weather_1900-2018.txt - Tab seperated file of Christmas Bird Count events going back to 1900. Each row represents a single count in a given year. Data Dictonary can be found here: http://www.audubon.org/sites/default/files/documents/cbc_report_field_definitions_2013.pdf

### Output Files
This is where you list any files or artifcats this notebook produces. Please also give a short summary of the data 

DataFiles should be named after the notebook when possible, ie this notebook might produce the file 0.0-rec-example-notebook-with-header.csv 


## Steps or Proceedures in the notebook 
This is where you list what you did in the notebook. The goal is that someone who has never coded in their life can understand what you did. Pro Tip -- if you are working off of an Github issue, this is a great place to put those check boxes! 
- Create an Example Header 
- Create an example of Loading Data 


## Where the Data will Be Saved 
All data for this project will be saved in Google Drive. To start experimenting with data, download the folder hear and put it into your data folder.
https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I

The path should look like this: 
audubon-cbc/data/Cloud_Data/<DATA FILE>

In [1]:
# Imports 
import pandas as pd

## This is a Section Header: How to Get Data For the Project
Data for this project is saved in Google Drive. You can start using this data by following the steps below
1. Clone the github repo (which I'm assuming you've done since your reading this)
2. Go to this folder and download it: https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I
3. Paste it into the data folder in your local repository. The path should look like this: audubon-cbc/data/Cloud_Data/
4. Voila! You have data. Lets test it.

In [5]:
# ALL File Paths should be declared at the TOP of the notebook
PATH_TO_RAW_CBC_DATA = "../data/Cloud_Data/1.3-rec-connecting-fips-ecosystem-data.txt"

In [6]:
raw_data = pd.read_csv(PATH_TO_RAW_CBC_DATA, encoding = "ISO-8859-1", sep="\t", compression='gzip')

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (62,64,65,71,73,75,77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
raw_data.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,block_fips,county_fips,Ecosys_circle,Usgsid_sys_circle,Nlcd_code_circle,Nlcd_circle,Ecosys_station,Usgsid_sys_station,Nlcd_code_station,Nlcd_station
0,Hawai'i: Volcano N.P.,US-HI,19.4333,-155.2833,1955,1955-01-01,11.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hawai'i: Volcano N.P.,US-HI,19.4333,-155.2833,1956,1955-12-31,11.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hawai'i: Volcano N.P.,US-HI,19.4333,-155.2833,1968,1967-12-30,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hawai'i: Volcano N.P.,US-HI,19.4333,-155.2833,1968,1967-12-30,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hawai'i: Volcano N.P.,US-HI,19.4333,-155.2833,1968,1967-12-30,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Saving Data From a Notebook
If a notebook is used to clean or add to a dataset, the resulting dataset should be named after the notebook that produced it. For example the data produced by this notebook would be called 0.0-rec-example-notebook-with-header.csv 

## Getting your data into Google Drive. 
If you have data you think is ready to be added to the offical google drive data folder, your DA will ask you for the following infomation.

- Number of Rows in the Data
- Discription and name of any new feilds 